In [1]:
from chess import pgn
from thefuzz import process
import pandas as pd
from tqdm import tqdm
from collections import Counter

In [2]:
fields = ['White', 'WhiteElo', 'Black', 'BlackElo', 'Result', 'Date']
data = []
with tqdm(total=5611947) as pbar:
    with open('../data/caissabase.pgn') as database:
        head = pgn.read_headers(database)
        gidx = 0
        while head is not None:
            try: 
                row = [gidx] + [head[x] for x in fields]
                misc = ''
                if 'Event' in head: misc += head['Event']
                if 'Site' in head: misc += head['Site']
                row.append(misc)
                data.append(row)
            except: pass
            head = pgn.read_headers(database)
            pbar.update(1)
            gidx += 1

100%|█████| 5611947/5611947 [04:18<00:00, 21681.08it/s]


In [3]:
df = pd.DataFrame(data, columns=['GameIdx'] + fields + ['Misc'])

In [4]:
names = set(pd.concat([df['White'], df['Black']]))
all_names = sorted(names)
name_dict = dict()

for i, name in enumerate(tqdm(all_names)):
    name_dict[name] = name
    if name in names:
        s = max(0, i-50); e = min(len(all_names)-1, i+50)
        equiv = [x[0] for x in process.extract(name, all_names[s:e])]
        names.remove(name)
        for n in equiv: 
            name_dict[n] = name
            try: names.remove(n)
            except: pass

100%|█████████| 137774/137774 [03:54<00:00, 587.46it/s]


In [12]:
df['White'] = df['White'].apply(lambda x: name_dict[x])
df['Black'] = df['Black'].apply(lambda x: name_dict[x])
df = df.drop_duplicates()
df = df[~df['Misc'].str.contains('960|simul|Simul')]
df = df[~df['Result'].str.contains('\*')]

In [14]:
df.to_csv('../data/games.csv')